In [ ]:
from tifffile import imread, imsave
import os, re, sys, csv
import numpy as np
import matplotlib.pyplot as pyp
import cv2

sys.path.append('~/3D_IMC_paper/Python/python_3d_imc_tools')
from io_files import image_filepath_for_3D_stack,get_folder_order_from_file, get_image_stack_for_one_channel_csv
from image_processing import local_image_normalization_stack, create_pseudo_image_with_pca

### This script generates PCA based pseudoimages where PCA is calculated across all the channels
The PCA loading are used for each channel to so that maximal infomration is contained on one image for all the channels.
These images were used for testing different registration options with ImageJ REgisterVirtualStack plugin.

In [ ]:
input_base = "~/3D_model201710/"

stack_single_channel_tiff_folder = input_base + '3Dstack_single_channel_tiffs/'

#slice order
stack3d_order = input_base + "final_3D_stack_order_model201710.csv"

final3d_order = {}
reader = csv.reader(open(stack3d_order, 'r'))    

for k,v in reader:
    v= int(v)
    #k = k.replace('.ome.tiff','')
    final3d_order[k] = v


# OUTPUT: single chanel registration stack to check the alignment of the images
singleChanel_dir = input_base + "PseudoPCA"

In [ ]:
sorted_dir = get_folder_order_from_file(stack_single_channel_tiff_folder,final3d_order)

In [ ]:
slice_order = 0

if not os.path.exists(singleChanel_dir):
    os.mkdir(singleChanel_dir)

for e in sorted_dir:

    channel_dir = os.path.join(stack_single_channel_tiff_folder, e)
    img_list = image_filepath_for_3D_stack(channel_dir, tiff = False)

    img = imread(img_list, pattern=None)
    print(img.shape)
    img = np.array(img, dtype=np.float32)
    img = local_image_normalization_stack(img, zscore = True, quant_norm = 0.99)
    #by default only calculates the first PCA that explains max variance
    pseudo_img = create_pseudo_image_with_pca(img)

    pseudo_img = np.array(pseudo_img, dtype=np.float32)

    outfile = '{0:04}'.format((slice_order+1)) + '_pseudoPCA.tiff'
    print(outfile)
    imsave(os.path.join(singleChanel_dir, outfile), pseudo_img, compress=False)

    slice_order = slice_order + 1


#### End of Notebook